7/3/2019

This notebook is for the baseline experiments for predicting the condition the speaker is in. 

The baseline model will take an utterance as an input and perform a three-way prediction task. The choices for the prediction are "close"--all colors are similar, "split"--the target is similar to one distractor and disimilar to another, and "far", the target is disimilar from both distractors which in turn are disimilar to each other.

In [1]:
# so we can access classes from parent directory
import sys
sys.path.append("..")

In [21]:
from monroe_data import MonroeData, MonroeDataEntry, Color # for loading in training data
import caption_featurizers                              # for getting caption representations
import color_featurizers                                # for getting color representations
from experiment import FeatureHandler                   # for combining caption and color features

from models import PytorchModel, ConditionPredictor # model base that handles training / evaluation

In [119]:
import torch
import torch.nn as nn
import numpy as np
from sklearn.metrics import classification_report, accuracy_score

In [124]:
import importlib
import models
importlib.reload(models)
from models import PytorchModel, ConditionPredictor # model base that handles training / evaluation

In [3]:
# get data
train_data = MonroeData("../data/csv/train_corpus_monroe.csv", "../data/entries/train_entries_monroe.pkl")
dev_data = MonroeData("../data/csv/dev_corpus_monroe.csv", "../data/entries/dev_entries_monroe.pkl")

In [5]:
# define feature functions
caption_phi_character = caption_featurizers.CaptionFeaturizer(tokenizer = caption_featurizers.CharacterTokenizer)

# we use a color featurizer because the code was designed to always use both color and text features. These 
# features will just be ignored in the model code
color_phi = color_featurizers.ColorFeaturizer(color_featurizers.color_phi_fourier, "hsv", normalized=True)

In [13]:
# we define a mapping from condition to index from closest to farthest
condition_to_idx = {"close": 0, "split": 1, "far":2}

# condition predictor's target is to predict what color condition the participants were put in
def condition_predictor_target(data_entry):
    return condition_to_idx[data_entry.condition]

# pass in train and dev data, our caption and color feature functions, function for turning an element of our data
# (train or dev) into the target, we don't care about the colors at all, but the feature handler expects them.
# We set randomized_colors to false because our target function shouldn't need to take a color index
feature_handler = FeatureHandler(train_data, dev_data, caption_phi_character, color_phi, target_fn=condition_predictor_target, randomized_colors=False)

In [14]:
X_train = feature_handler.train_features()
y_train = feature_handler.train_targets()

In [112]:
# Now let's define the model:
class BaselineConditionPredictor(nn.Module):
    
    def __init__(self, vocab_size, embed_dim, speaker_hidden_dim, num_conditions=3):
        super(BaselineConditionPredictor, self).__init__()
        
        self.embed = nn.Embedding(vocab_size, embed_dim)
        
        self.speaker_lstm = nn.LSTM(embed_dim, speaker_hidden_dim, batch_first=True)
        self.linear = nn.Linear(speaker_hidden_dim, num_conditions)
        self.logsoftmax = nn.LogSoftmax(dim=1)
        
        
    def forward(self, color, caption):
        # disregard color because we don't care about it
        
        inputs = self.embed(caption)
        hidden, _ = self.speaker_lstm(inputs)
        last_hidden = hidden[:, -1, :]
        output_lin = self.linear(last_hidden)
        outputs = self.logsoftmax(output_lin)
        return outputs # let's just use the last embedding for the predictions

In [77]:
condition_predictor_character = ConditionPredictor(BaselineConditionPredictor, optimizer=torch.optim.Adam, lr=0.004, num_epochs=5)
condition_predictor_character.init_model(vocab_size=caption_phi_character.caption_indexer.size, 
                                         embed_dim=100, speaker_hidden_dim=100)

In [78]:
condition_predictor_character.fit(X_train, y_train)

---EPOCH 0---
0m 0s (0:0 0.00%) 0.0012
0m 6s (0:1000 7.90%) 1.0522
0m 13s (0:2000 15.79%) 1.0662
0m 20s (0:3000 23.69%) 1.0597
0m 29s (0:4000 31.58%) 1.0321
0m 37s (0:5000 39.48%) 1.0621
0m 44s (0:6000 47.37%) 1.0399
0m 51s (0:7000 55.27%) 1.0307
0m 58s (0:8000 63.17%) 1.0688
1m 4s (0:9000 71.06%) 1.0150
1m 11s (0:10000 78.96%) 1.0470
1m 17s (0:11000 86.85%) 1.0667
1m 23s (0:12000 94.75%) 1.0538
AFTER EPOCH 2999 - AVERAGE VALIDATION LOSS: 1.0571353808045387
---EPOCH 1---
1m 31s (1:0 0.00%) 0.0012
1m 37s (1:1000 7.90%) 1.0336
1m 45s (1:2000 15.79%) 1.0613
1m 52s (1:3000 23.69%) 1.0620
2m 0s (1:4000 31.58%) 1.0409
2m 7s (1:5000 39.48%) 1.0716
2m 15s (1:6000 47.37%) 1.0226
2m 22s (1:7000 55.27%) 1.0158
2m 29s (1:8000 63.17%) 1.0848
2m 36s (1:9000 71.06%) 1.0537
2m 42s (1:10000 78.96%) 1.0333
2m 48s (1:11000 86.85%) 1.0339
2m 54s (1:12000 94.75%) 1.0540
AFTER EPOCH 2999 - AVERAGE VALIDATION LOSS: 1.0473164211710293
---EPOCH 2---
3m 3s (2:0 0.00%) 0.0008
3m 10s (2:1000 7.90%) 1.0477
3m 17s 

In [79]:
condition_predictor_character.save_model("../model/condition_predictor_character.params")

In [40]:
X_assess = feature_handler.test_features()
y_assess = feature_handler.test_targets()

In [47]:
condition_predictor_character.load_model("../model/condition_predictor_character.params")

In [80]:
predictions_character = condition_predictor_character.predict(X_assess)

[-1.7478129 -1.3159838 -0.5840502]
[-2.3660455 -1.7839618 -0.303571 ]
[-1.2228812 -1.0489761 -1.034728 ]
[-1.3406556 -0.9936053 -0.9994374]
[-2.189497   -1.9687753  -0.28981888]
[-1.2167933  -0.96872765 -1.1262224 ]
[-2.3660455 -1.7839618 -0.303571 ]
[-2.3660455 -1.7839618 -0.303571 ]
[-1.2186503 -0.9709107 -1.121984 ]
[-2.3660455 -1.7839618 -0.303571 ]
[-1.0800321 -1.2433364 -0.9888747]
[-1.2538704 -0.9882405 -1.0718606]
[-1.2711487 -1.0233942 -1.0213245]
[-2.189497   -1.9687753  -0.28981888]
[-1.0752101  -1.2395554  -0.99625164]
[-1.216706  -0.9686061 -1.1264443]
[-2.3795917  -1.5385686  -0.36712444]
[-2.662344   -1.7603421  -0.27676904]
[-1.2493832  -0.99510294 -1.068181  ]
[-2.5155637 -1.8459495 -0.2727182]
[-1.345469   -0.99478644 -0.99484676]
[-1.1131723  -1.2484732  -0.95570177]
[-2.3795917  -1.5385686  -0.36712444]
[-2.1442528  -1.7235308  -0.35039547]
[-1.0762403 -1.2403421 -0.9946845]
[-1.0123879 -1.1114081 -1.1791033]
[-1.3404921  -0.99405944 -0.9990971 ]
[-1.4208044 -0.9853

[-0.81994075 -1.0382165  -1.5825206 ]
[-1.4858708  -1.086327   -0.82956016]
[-1.6369196  -1.3771482  -0.59217227]
[-1.2622347 -1.080103  -0.9744004]
[-1.0915445 -1.0124642 -1.2007121]
[-0.81986266 -1.0379503  -1.5831466 ]
[-1.2228725  -0.97632277 -1.1119318 ]
[-1.218107   -0.96889466 -1.1248283 ]
[-1.4508487  -0.99412    -0.92740285]
[-1.4664223  -0.93486583 -0.97653997]
[-1.4660901  -0.93187416 -0.97987294]
[-2.3795917  -1.5385686  -0.36712444]
[-1.038727  -1.0340203 -1.2360808]
[-1.2953483 -1.0092018 -1.016974 ]
[-2.1442528  -1.7235308  -0.35039547]
[-0.8198785 -1.0378687 -1.5832534]
[-0.83225477 -1.0390424  -1.5552363 ]
[-2.662344   -1.7603421  -0.27676904]
[-1.8630877 -1.2943213 -0.5608505]
[-2.189497   -1.9687753  -0.28981888]
[-0.82580894 -1.0395093  -1.5678582 ]
[-1.2490747 -0.9974675 -1.0659001]
[-1.1255828 -1.1028838 -1.0682068]
[-2.1442528  -1.7235308  -0.35039547]
[-2.3795917  -1.5385686  -0.36712444]
[-1.8630877 -1.2943213 -0.5608505]
[-2.1442528  -1.7235308  -0.35039547]
[

[-1.4285569  -1.0290843  -0.90879047]
[-2.3660455 -1.7839618 -0.303571 ]
[-1.017827  -0.9708915 -1.347553 ]
[-0.84398365 -1.0682447  -1.4854753 ]
[-1.6369196  -1.3771482  -0.59217227]
[-1.2712724 -1.020258  -1.0243673]
[-1.3616719  -1.1150126  -0.87741417]
[-1.4285569  -1.0290843  -0.90879047]
[-1.0755647 -1.0751712 -1.1467875]
[-2.3795917  -1.5385686  -0.36712444]
[-1.0453545 -0.9722612 -1.3085709]
[-1.3794013 -1.2076778 -0.799886 ]
[-1.463311   -0.9306163  -0.98290914]
[-1.0453902 -0.9722612 -1.3085239]
[-1.8630877 -1.2943213 -0.5608505]
[-1.2168963 -0.9686432 -1.126227 ]
[-1.2709011 -1.0061768 -1.0390033]
[-0.99085987 -1.0571878  -1.2682935 ]
[-2.1442528  -1.7235308  -0.35039547]
[-1.9264256 -1.6213034 -0.4205415]
[-2.5155637 -1.8459495 -0.2727182]
[-1.7934551 -1.140372  -0.6656961]
[-1.27745   -1.4066553 -0.7417252]
[-2.4426932 -1.958077  -0.2588433]
[-1.6369196  -1.3771482  -0.59217227]
[-2.3660455 -1.7839618 -0.303571 ]
[-1.3877192 -1.2269788 -0.7826802]
[-0.8295465 -1.0397956 -1

[-1.288087  -1.0258365 -1.0059266]
[-1.249067   -0.99744654 -1.0659289 ]
[-2.189497   -1.9687753  -0.28981888]
[-1.8630877 -1.2943213 -0.5608505]
[-1.8630877 -1.2943213 -0.5608505]
[-1.2736535 -1.1387703 -0.9163409]
[-1.0609115 -1.0536181 -1.1868341]
[-1.3094013 -1.0140648 -1.0016395]
[-2.3660455 -1.7839618 -0.303571 ]
[-1.4852024 -1.3265032 -0.6769986]
[-1.7809217 -1.4314935 -0.523294 ]
[-1.4852024 -1.3265032 -0.6769986]
[-1.3867068 -1.1649377 -0.8251688]
[-1.9264256 -1.6213034 -0.4205415]
[-2.662344   -1.7603421  -0.27676904]
[-1.4852024 -1.3265032 -0.6769986]
[-2.3795917  -1.5385686  -0.36712444]
[-2.3660455 -1.7839618 -0.303571 ]
[-2.3660455 -1.7839618 -0.303571 ]
[-2.01531    -1.6818885  -0.38463572]
[-1.1837186 -1.1153889 -1.0049195]
[-1.0506017 -0.9752315 -1.2976693]
[-1.3426945 -1.192847  -0.8312548]
[-1.6027699  -1.200062   -0.69818705]
[-2.1801333 -1.7877433 -0.3290094]
[-1.8193091 -1.2939634 -0.5732679]
[-1.4514549 -1.1153283 -0.8256207]
[-2.3795917  -1.5385686  -0.36712444]

[-1.340225  -0.9936483 -0.9997003]
[-1.2882123 -1.1707139 -0.881669 ]
[-1.3401432  -0.9935681  -0.99983925]
[-1.3457704 -1.0175192 -0.9724106]
[-1.3777623 -1.0062432 -0.9616334]
[-1.2674143 -1.0081223 -1.0397646]
[-1.2488447  -0.99455774 -1.0692173 ]
[-0.82998127 -1.0669703  -1.5146129 ]
[-1.0391161 -1.0345155 -1.2350018]
[-1.2882123 -1.1707139 -0.881669 ]
[-1.0391161 -1.0345155 -1.2350018]
[-1.2523069  -0.98727065 -1.0742228 ]
[-1.2808242 -1.1689974 -0.8879136]
[-1.2085779 -0.8879516 -1.2382865]
[-1.0153837 -0.9309887 -1.4123392]
[-0.91981024 -0.93436044 -1.5674889 ]
[-1.0754204 -1.0746931 -1.1474564]
[-1.1081288 -1.044807  -1.1455024]
[-2.017219  -1.6879745 -0.3826062]
[-1.4322246 -1.0288286 -0.9068419]
[-2.017219  -1.6879745 -0.3826062]
[-1.7478129 -1.3159838 -0.5840502]
[-1.8630877 -1.2943213 -0.5608505]
[-2.662344   -1.7603421  -0.27676904]
[-2.3660455 -1.7839618 -0.303571 ]
[-1.4322246 -1.0288286 -0.9068419]
[-1.5319409 -1.0601026 -0.826761 ]
[-1.2624154 -0.9994466 -1.0528347]
[-

...

[-2.3660455 -1.7839618 -0.303571 ]
[-2.3795917  -1.5385686  -0.36712444]
[-1.7809217 -1.4314935 -0.523294 ]
[-1.7809217 -1.4314935 -0.523294 ]
[-2.3795917  -1.5385686  -0.36712444]
[-1.0401464 -1.0343028 -1.2340099]
[-1.0481198 -1.035303  -1.2232069]
[-1.2508459  -0.99545753 -1.0665814 ]
[-2.3660455 -1.7839618 -0.303571 ]
[-1.8630877 -1.2943213 -0.5608505]
[-1.2897506 -1.1547577 -0.8927533]
[-1.1549003 -0.9837214 -1.1679817]
[-1.8630877 -1.2943213 -0.5608505]
[-1.5318878 -1.0600581 -0.8268226]
[-2.3660455 -1.7839618 -0.303571 ]
[-1.8630877 -1.2943213 -0.5608505]
[-1.7478129 -1.3159838 -0.5840502]
[-1.6123599  -1.3264189  -0.62519425]
[-1.0580741 -1.0600808 -1.1827056]
[-2.3795917  -1.5385686  -0.36712444]
[-1.371272  -1.0233305 -0.9497957]
[-1.3428216 -0.9950709 -0.9964306]
[-1.1056452 -1.096616  -1.0936149]
[-1.2167338 -0.9686074 -1.126417 ]
[-1.8630877 -1.2943213 -0.5608505]
[-2.3795917  -1.5385686  -0.36712444]
[-0.82812524 -1.0380725  -1.565441  ]
[-1.6369196  -1.3771482  -0.592172

[-2.3795917  -1.5385686  -0.36712444]
[-1.8630877 -1.2943213 -0.5608505]
[-1.5791718 -1.0146241 -0.8409153]
[-1.2488432 -0.9945607 -1.0692153]
[-1.7478129 -1.3159838 -0.5840502]
[-1.0729107 -1.0735087 -1.1514393]
[-1.8630877 -1.2943213 -0.5608505]
[-1.2679191  -1.1445024  -0.91578823]
[-1.9093685 -1.2386196 -0.5761785]
[-2.3660455 -1.7839618 -0.303571 ]
[-1.3211894  -1.016346   -0.99083304]
[-1.2492015 -0.9949464 -1.068501 ]
[-1.3410084  -0.99382734 -0.99896336]
[-1.654784  -1.1692014 -0.6966496]
[-1.9093685 -1.2386196 -0.5761785]
[-2.3660455 -1.7839618 -0.303571 ]
[-1.3269106 -1.0986995 -0.9127965]
[-2.3660455 -1.7839618 -0.303571 ]
[-1.2181113 -0.9689006 -1.1248176]
[-1.3410084  -0.99382734 -0.99896336]
[-1.4278497  -1.0284823  -0.90974545]
[-2.3660455 -1.7839618 -0.303571 ]
[-1.8630877 -1.2943213 -0.5608505]
[-1.7809217 -1.4314935 -0.523294 ]
[-1.0929248 -1.0843226 -1.118913 ]
[-1.2508459 -0.9954567 -1.0665824]
[-2.3660455 -1.7839618 -0.303571 ]
[-0.8199236 -1.0380147 -1.582905 ]
[-

[-1.091925  -1.1843228 -1.0258912]
[-1.468147   -0.9356785  -0.97463906]
[-0.82812524 -1.0380725  -1.565441  ]
[-1.4594041 -1.1238656 -0.8150792]
[-1.0742472 -1.0745318 -1.1488926]
[-1.2565778 -1.0192069 -1.0370681]
[-2.662344   -1.7603421  -0.27676904]
[-1.1175125 -1.2507045 -0.950353 ]
[-1.2320596 -1.0596533 -1.0168493]
[-2.3660455 -1.7839618 -0.303571 ]
[-1.2438785 -1.0633993 -1.0038772]
[-1.3400077  -0.99349487 -1.0000093 ]
[-1.6556755 -1.1699848 -0.6958201]
[-1.0929248 -1.0843226 -1.118913 ]
[-1.0929248 -1.0843226 -1.118913 ]
[-1.4183145  -1.1894034  -0.79081416]
[-1.2167355 -0.9686601 -1.1263539]
[-1.2137781 -1.0945162 -0.9990591]
[-2.189497   -1.9687753  -0.28981888]
[-1.2137781 -1.0945162 -0.9990591]
[-1.8630877 -1.2943213 -0.5608505]
[-1.2628214 -1.0022124 -1.0495971]
[-1.654784  -1.1692014 -0.6966496]
[-1.5558105  -0.98497057 -0.8781967 ]
[-1.5005671 -1.0458293 -0.8542645]
[-1.2137781 -1.0945162 -0.9990591]
[-1.262977  -1.0015095 -1.0502092]
[-1.7475793  -1.4800913  -0.513850

[-1.045593   -0.98494846 -1.2907729 ]
[-1.8630877 -1.2943213 -0.5608505]
[-2.3795917  -1.5385686  -0.36712444]
[-1.7809217 -1.4314935 -0.523294 ]
[-1.7478129 -1.3159838 -0.5840502]
[-1.250905   -0.99852073 -1.0632539 ]
[-1.2181113 -0.9689006 -1.1248176]
[-1.3406054 -0.994128  -0.9989476]
[-1.2874527 -1.0264766 -1.0057777]
[-2.01531    -1.6818885  -0.38463572]
[-1.061007  -1.0703456 -1.1679529]
[-0.9499574 -1.0620921 -1.3185956]
[-0.9608426 -1.0859481 -1.2735087]
[-2.3795917  -1.5385686  -0.36712444]
[-1.257668  -1.0776856 -0.9800258]
[-2.1442528  -1.7235308  -0.35039547]
[-2.1442528  -1.7235308  -0.35039547]
[-2.662344   -1.7603421  -0.27676904]
[-2.862689   -2.1074557  -0.19682026]
[-2.662344   -1.7603421  -0.27676904]
[-1.2623627 -0.9447056 -1.1140671]
[-2.662344   -1.7603421  -0.27676904]
[-1.6369196  -1.3771482  -0.59217227]
[-1.6369196  -1.3771482  -0.59217227]
[-1.6369196  -1.3771482  -0.59217227]
[-1.2624539 -0.9995332 -1.0527122]
[-0.83018994 -1.0669084  -1.5142963 ]
[-2.189497

[-1.860491   -1.2352078  -0.59125966]
[-1.351573   -1.112181   -0.88594055]
[-1.6369196  -1.3771482  -0.59217227]
[-1.6369196  -1.3771482  -0.59217227]
[-1.3427467  -0.99501014 -0.9965445 ]
[-0.82812524 -1.0380725  -1.565441  ]
[-1.3868785 -1.1650572 -0.8249858]
[-1.4311757 -1.0446095 -0.8936903]
[-1.6369196  -1.3771482  -0.59217227]
[-0.8870154 -1.0625985 -1.4165058]
[-2.01531    -1.6818885  -0.38463572]
[-1.5319409 -1.0601026 -0.826761 ]
[-1.3427467  -0.99501014 -0.9965445 ]
[-1.8630877 -1.2943213 -0.5608505]
[-2.3795917  -1.5385686  -0.36712444]
[-2.3660455 -1.7839618 -0.303571 ]
[-2.1442528  -1.7235308  -0.35039547]
[-1.227369   -0.97129583 -1.1136898 ]
[-1.3926766  -1.0396873  -0.92123985]
[-1.0455415  -0.97227997 -1.3083007 ]
[-1.7809217 -1.4314935 -0.523294 ]
[-1.6369196  -1.3771482  -0.59217227]
[-2.01531    -1.6818885  -0.38463572]
[-1.1158557 -1.2709613 -0.9369952]
[-1.1255828 -1.1028838 -1.0682068]
[-2.1442528  -1.7235308  -0.35039547]
[-1.2491186  -0.99516016 -1.0683403 ]
[

In [89]:
y_hat_character = np.argmax(predictions_character, axis=1)

In [96]:
sum(y_hat_character == 2)/len(y_hat_character)

0.5469687300574346

In [118]:
sum(y_hat_character == 0)

2307

In [117]:
sum(y_assess == 0)

5171

In [94]:
print(classification_report(y_hat_character, y_assess, target_names=['close', 'split', 'far']))

              precision    recall  f1-score   support

       close       0.21      0.47      0.29      2307
       split       0.35      0.38      0.37      4792
         far       0.77      0.48      0.59      8571

   micro avg       0.45      0.45      0.45     15670
   macro avg       0.45      0.44      0.42     15670
weighted avg       0.56      0.45      0.48     15670



In [123]:
accuracy_score(y_hat_character, y_assess)

0.4469049138481174

So that's interesting, naively doing this seems to mean that we are worse than chance... I think I must be doing something incorrectly. This seems unreasonably bad. I wonder if using bpe tokens as features would help at all (or even word features)

In [102]:
caption_phi_endings = caption_featurizers.CaptionFeaturizer(tokenizer = caption_featurizers.EndingTokenizer)

# we use a color featurizer because the code was designed to always use both color and text features. These 
# features will just be ignored in the model code
color_phi = color_featurizers.ColorFeaturizer(color_featurizers.color_phi_fourier, "hsv", normalized=True)
feature_handler_endings = FeatureHandler(train_data, dev_data, caption_phi_endings, color_phi, target_fn=condition_predictor_target, randomized_colors=False)

In [103]:
X_train_endings = feature_handler_endings.train_features()
y_train_endings = feature_handler_endings.train_targets()

In [104]:
condition_predictor_endings = ConditionPredictor(BaselineConditionPredictor, optimizer=torch.optim.Adam, lr=0.004, num_epochs=5)
condition_predictor_endings.init_model(vocab_size=caption_phi_endings.caption_indexer.size, 
                                         embed_dim=100, speaker_hidden_dim=100)

In [106]:
condition_predictor_endings.fit(X_train_endings, y_train_endings)

---EPOCH 0---
0m 0s (0:0 0.00%) 0.0012
0m 3s (0:1000 7.90%) 1.0270
0m 8s (0:2000 15.79%) 1.0476
0m 13s (0:3000 23.69%) 1.0439
0m 20s (0:4000 31.58%) 1.0302
0m 26s (0:5000 39.48%) 1.0205
0m 33s (0:6000 47.37%) 0.9947
0m 39s (0:7000 55.27%) 0.9759
0m 46s (0:8000 63.17%) 1.0362
0m 52s (0:9000 71.06%) 0.9631
0m 59s (0:10000 78.96%) 0.9812
1m 5s (0:11000 86.85%) 1.0094
1m 12s (0:12000 94.75%) 0.9997
AFTER EPOCH 2999 - AVERAGE VALIDATION LOSS: 1.012975144247214
---EPOCH 1---
1m 19s (1:0 0.00%) 0.0013
1m 26s (1:1000 7.90%) 0.9891
1m 33s (1:2000 15.79%) 1.0179
1m 40s (1:3000 23.69%) 1.0267
1m 48s (1:4000 31.58%) 1.0030
1m 55s (1:5000 39.48%) 0.9903
2m 2s (1:6000 47.37%) 0.9831
2m 8s (1:7000 55.27%) 0.9763
2m 15s (1:8000 63.17%) 1.0393
2m 22s (1:9000 71.06%) 0.9746
2m 29s (1:10000 78.96%) 0.9808
2m 36s (1:11000 86.85%) 1.0075
2m 43s (1:12000 94.75%) 1.0030
AFTER EPOCH 2999 - AVERAGE VALIDATION LOSS: 1.0154820242524147
---EPOCH 2---
2m 49s (2:0 0.00%) 0.0011
2m 56s (2:1000 7.90%) 0.9831
3m 4s (2

In [107]:
X_assess_endings = feature_handler_endings.test_features()
y_assess_endings = feature_handler_endings.test_targets()

In [108]:
predictions_endings = condition_predictor_endings.predict(X_assess_endings)
y_hat_endings = np.argmax(predictions_endings, axis=1)

In [109]:
print(classification_report(y_hat_endings, y_assess_endings, target_names=['close', 'split', 'far']))

              precision    recall  f1-score   support

       close       0.39      0.55      0.46      3713
       split       0.39      0.42      0.40      4863
         far       0.74      0.55      0.63      7094

   micro avg       0.51      0.51      0.51     15670
   macro avg       0.51      0.50      0.50     15670
weighted avg       0.55      0.51      0.52     15670



In [122]:
accuracy_score(y_hat_endings, y_assess_endings)

0.5068283343969369

In [110]:
# now for the subword features:
caption_phi_subword = caption_featurizers.CaptionFeaturizer(tokenizer = caption_featurizers.SentencePieceTokenizer)

feature_handler_subword = FeatureHandler(train_data, dev_data, caption_phi_subword, color_phi, target_fn=condition_predictor_target, randomized_colors=False)


In [111]:
X_train_subword = feature_handler_subword.train_features()
y_train_subword = feature_handler_subword.train_targets()

In [113]:
condition_predictor_subword = ConditionPredictor(BaselineConditionPredictor, optimizer=torch.optim.Adam, lr=0.004, num_epochs=5)
condition_predictor_subword.init_model(vocab_size=caption_phi_subword.caption_indexer.size, 
                                         embed_dim=100, speaker_hidden_dim=100)

In [114]:
condition_predictor_subword.fit(X_train_subword, y_train_subword)

---EPOCH 0---
0m 0s (0:0 0.00%) 0.0011
0m 3s (0:1000 7.90%) 1.0670
0m 7s (0:2000 15.79%) 1.0381
0m 11s (0:3000 23.69%) 1.0534
0m 17s (0:4000 31.58%) 1.0503
0m 21s (0:5000 39.48%) 1.0532
0m 26s (0:6000 47.37%) 1.0210
0m 29s (0:7000 55.27%) 1.0037
0m 33s (0:8000 63.17%) 1.0616
0m 37s (0:9000 71.06%) 0.9924
0m 41s (0:10000 78.96%) 1.0199
0m 44s (0:11000 86.85%) 1.0379
0m 48s (0:12000 94.75%) 1.0080
AFTER EPOCH 2999 - AVERAGE VALIDATION LOSS: 1.0338063392241796
---EPOCH 1---
0m 53s (1:0 0.00%) 0.0014
0m 56s (1:1000 7.90%) 1.0313
1m 0s (1:2000 15.79%) 1.0315
1m 4s (1:3000 23.69%) 1.0488
1m 9s (1:4000 31.58%) 1.0295
1m 13s (1:5000 39.48%) 1.0290
1m 17s (1:6000 47.37%) 1.0106
1m 20s (1:7000 55.27%) 0.9888
1m 24s (1:8000 63.17%) 1.0438
1m 28s (1:9000 71.06%) 0.9848
1m 32s (1:10000 78.96%) 0.9985
1m 35s (1:11000 86.85%) 1.0216
1m 38s (1:12000 94.75%) 1.0217
AFTER EPOCH 2999 - AVERAGE VALIDATION LOSS: 1.0292375600536665
---EPOCH 2---
1m 43s (2:0 0.00%) 0.0009
1m 47s (2:1000 7.90%) 1.0133
1m 52s 

In [115]:
X_assess_subword = feature_handler_subword.test_features()
y_assess_subword = feature_handler_subword.test_targets()

In [116]:
predictions_subword = condition_predictor_subword.predict(X_assess_subword)
y_hat_subword = np.argmax(predictions_subword, axis=1)
print(classification_report(y_hat_subword, y_assess_subword, target_names=['close', 'split', 'far']))

              precision    recall  f1-score   support

       close       0.29      0.52      0.37      2852
       split       0.35      0.40      0.37      4541
         far       0.77      0.49      0.60      8277

   micro avg       0.47      0.47      0.47     15670
   macro avg       0.47      0.47      0.45     15670
weighted avg       0.56      0.47      0.49     15670



In [120]:
accuracy_score(y_hat_subword, y_assess_subword)

0.4696873005743459

In [130]:
# I want to compare this with the pragmatic, so I'm going to use endings

predictions_training_subword = condition_predictor_subword.predict(X_train_subword)
y_hat_training_subword = np.argmax(predictions_training_subword, axis=1)
print(classification_report(y_hat_training_subword, y_train_subword, target_names=['close', 'split', 'far']))

              precision    recall  f1-score   support

       close       0.28      0.53      0.36      2727
       split       0.31      0.40      0.35      4071
         far       0.79      0.47      0.59      8867

   micro avg       0.46      0.46      0.46     15665
   macro avg       0.46      0.47      0.43     15665
weighted avg       0.58      0.46      0.49     15665



In [131]:
y_hat_training_subword[:10]

array([0, 2, 1, 1, 2, 2, 2, 2, 1, 2])

In [132]:
import pickle

with open("../results/condition_prediction_baseline_training_subword.pkl", "wb") as file:
    pickle.dump(y_hat_training_subword, file)